In [214]:
import pandas as pd
from pandas import option_context

import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split

import re
import string

In [215]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tawneykirkland/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Main notebook for pre-processing tweets

### Prepare dataframe

In [216]:
df = pd.read_csv('custom_out2.csv')
df.dropna(axis=1,inplace=True)

/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [217]:
## Filter down to non-Revolut tweets (intention is to understand tweets from customers)
df_eng = df[(df['username'] != 'revolutapp') & (df['language'] == 'en')]

In [218]:
# Train test split
x_tr, x_test= train_test_split(df_eng,test_size=.20, random_state=10)

__Note the x_tr will need to be further split into train and validate__

### Clean tweet column

In [219]:
def clean_text(text):
    '''Make text lowercase, remove punctuation, remove reference to @revolutapp,remove links and mentions'''
    text = text.lower()
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)
    text = re.sub('[0-9\n]',' ',text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('revolutapp', '', text)
    
    return text

x_tr['tweet'] = x_tr.tweet.map(clean_text)

<ipython-input-219-e48a59a1314a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_tr['tweet'] = x_tr.tweet.map(clean_text)


In [220]:
## Remove emojis
x_tr['tweet'] = x_tr['tweet'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

<ipython-input-220-8c7353a8d011>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_tr['tweet'] = x_tr['tweet'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))


In [221]:
with option_context('display.max_colwidth', 600):
    display(x_tr.sample(10))

,id,conversation_id,created_at,date,time,timezone,user_id,username,tweet,language,...,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,video,reply_to
186794,1243161087308107776,1243150533747187714,2020-03-26 09:01:14 EDT,2020-03-26,09:01:14,-500,1243120267330826240,hosnibenalaya1,i m in france and they blocked me since last summer and now i don t know if my account is only blocked or descativated and the money on i need my money i waited for long time now i need my money you cant decide where to transfere my money without letting meknow,en,...,[],0,0,0,[],[],https://twitter.com/HosniBenAlaya1/status/1243161087308107776,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}, {'screen_name': 'TrussellTrust', 'name': 'The Trussell Trust', 'id': '76972169'}]"
83366,1305143513802330112,1304783066301435904,2020-09-13 09:57:16 EDT,2020-09-13,09:57:16,-500,1234348893829287936,bbrawbtm23,oh and they have blocked me not helped me,en,...,[],0,1,1,[],[],https://twitter.com/bbrawbtm23/status/1305143513802330112,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}, {'screen_name': 'behindyournails', 'name': 'Salomé', 'id': '1239262079107960832'}]"
139122,1271781690923507712,1271508444550500352,2020-06-13 08:29:18 EDT,2020-06-13,08:29:18,-500,42584365,asvanevik,if youre mostly outside eu it pays for itself or even more up until now higher atm limits insurance early access to features eg commodities trading,en,...,[],0,0,1,[],[],https://twitter.com/ASvanevik/status/1271781690923507712,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}]"
108758,1289717890837409792,1288036035503493121,2020-08-01 20:21:21 EDT,2020-08-01,20:21:21,-500,146800154,miikolas,same here,en,...,[],0,0,0,[],[],https://twitter.com/Miikolas/status/1289717890837409792,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}]"
67165,1318787361363398656,1318585928735248385,2020-10-21 01:33:03 EDT,2020-10-21,01:33:03,-500,31138303,luplante,why is revolut not helping the issue ive had for days ive tried everything to get in touch,en,...,[],1,0,0,[],[],https://twitter.com/LuPlante/status/1318787361363398656,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}]"
212555,1227154160073748480,1227154160073748480,2020-02-11 03:55:25 EST,2020-02-11,03:55:25,-500,1081552369450651648,sam44646874,i need help accessing my funds,en,...,[],1,0,0,[],[],https://twitter.com/Sam44646874/status/1227154160073748480,False,0,[]
85386,1303682038877294593,1303651019105144832,2020-09-09 09:09:53 EDT,2020-09-09,09:09:53,-500,19705250,sufranke,juhuu danke michael thx and an award for perfectly hiding the navigation to chat its not up to me of course,en,...,[],1,0,2,[],[],https://twitter.com/sufranke/status/1303682038877294593,False,0,"[{'screen_name': 'budimanjob', 'name': 'Michael Job', 'id': '373606930'}, {'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}]"
123408,1280451690572001282,941717790679031808,2020-07-07 06:40:47 EDT,2020-07-07,06:40:47,-500,1082140986476847104,mtgox101,its not worth much for all the reasons he laid out be like cashapp not robinhood,en,...,[],0,0,0,[],[],https://twitter.com/MtGox101/status/1280451690572001282,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}, {'screen_name': 'VitalikButerin', 'name': 'vitalik.eth', 'id': '295218901'}]"
189439,1242134967389470721,1242134967389470721,2020-03-23 13:03:48 EDT,2020-03-23,13:03:48,-500,951138206199803904,cracked_account,can you dm me i need help,en,...,[],1,0,0,[],[],https://twitter.com/CRACKED_ACCOUNT/status/1242134967389470721,False,0,[]
84146,1304397997904715779,1304396251195211776,2020-09-11 08:34:51 EDT,2020-09-11,08:34:51,-500,2505539400,washedbatch,i hope this means cheaper steak for the tl,en,...,[],0,0,2,[],[],https://twitter.com/WashedBatch/status/1304397997904715779,False,0,"[{'screen_name': 'm_chael', 'name': 'Michael from

In [222]:
## Remove known observations that will not add value
x_tr = x_tr[(x_tr['tweet'] != ' followed') & (x_tr['tweet'] != 'followed ')]

### Word tokenization

In [223]:
# x_tr['tokens'] = x_tr.tweet.apply(nltk.word_tokenize)

In [224]:
# with option_context('display.max_colwidth', 600):
#     display(x_tr.sample(5))

#### Note: How to find and correct mispelled words?
Where in the pipeline is best place to do this?

In [225]:
remove_misspelled = (lambda x: 
                 x.replace('seriusly', 'seriously')
                 .replace('incomewhich', 'income which')
                 .replace('revolutteam', 'revolut team')
                 .replace('revolutuserswantscrypto', 'revolut users want crypto')
                 .replace('chatwhere', 'chat where')
                 .replace('loungesthat', 'lounges that')
                 .replace('moreexcuses', 'more excuses')
                 .replace('relovut', 'revolut')
                 .replace('nopromisedphonecall', 'no promised phone call')
                 .replace('complient', 'compliant')
                 .replace('hapenned', 'happened')
                 .replace('problembut', 'problem but')
                 .replace('sitesstill', 'site still')
                 .replace('moneylaundring', 'money laundering')
                 .replace('changeslogin', 'changes login')
                 .replace('transactioned', 'transaction')
                 .replace('reasonsprovided', 'reasons provided')
                 .replace('blockingaccount', 'blocking account')
                 .replace('whrite', 'right')
                 .replace('againshame', 'again shame')
                 .replace('moneygood', 'money good')
                 .replace('premim', 'premium')
                 .replace('pinwont', 'pinwont')
                 .replace('resetwont', 'reset wont')
                 .replace('resetfrustrating', 'reset frustrating')
                 .replace('moneywithout', 'money without')
                 .replace('conserns', 'concerns')
                 .replace('exchanses', 'exchanges')
                 .replace('plattform', 'platform')
                 .replace('informaton', 'information')
                 .replace('generatingsaving', 'generating saving')
                 .replace('breakthesilence', 'break the silence')
                 .replace('monthsampnosupportampnorefund', 'months and no support and no refund')
                 .replace('customerservicemissing', 'customer service missing')
                 .replace('verifiacationcould', 'verification could')
                 .replace('regulatinons', 'regulations')
                 .replace('fraudulence', 'fraudulent')
                 .replace('imsurqnce', 'insurance')
                 .replace('consumercomplaint', 'consumer complaint'))

x_tr['tweet'] = x_tr.tweet.map(remove_misspelled)

### Lemmatize, Tokenize and count words

Note that this is separate to tokenization above - what is correct in terms of process?

In [206]:
from nltk.util import ngrams

from collections import Counter

from nltk.corpus import stopwords

In [207]:
stop = stopwords.words('english')
stop.extend(['ive', 'im', 'itd', 'youre', 'hi', 'hello', 'hey', 'eg', 'l', 'h','w','v','u'])

In [227]:
lemma = nltk.stem.WordNetLemmatizer()
counter = Counter()

for tweet in x_tr['tweet']:
    tweet = lemma.lemmatize(tweet)
    tweet = TextBlob(tweet).words  # tokenize words
    tweet = [w for w in tweet if w not in stop]   
    counter += Counter(tweet)

In [228]:
len(counter)

32013

In [230]:
for phrase, count in counter.most_common(100):
    print('%20s %i' % ("".join(phrase), count))

             account 27822
             revolut 16391
                 app 15532
               money 14068
              please 12584
                help 11192
                 get 9938
                cant 9386
                chat 8721
               still 7562
                card 7437
                need 7256
                dont 6027
              thanks 5811
                 one 5737
              access 5612
             support 5587
                bank 5181
             blocked 4974
                days 4777
             waiting 4673
                back 4587
                like 4424
             service 4347
              locked 4298
                 use 4291
            customer 4186
                 new 4160
               since 4156
             contact 4101
               agent 3851
                time 3762
               would 3731
               phone 3672
                  dm 3622
             someone 3415
                 got 3381
               issue 3314
      